In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml import PipelineModel
from pyspark.ml.tuning import CrossValidatorModel
from cassandra.cluster import Cluster
from pyspark.sql.functions import col, when
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

In [49]:
spark = SparkSession.builder.getOrCreate()

# Load the saved pipelineFit model
pipelineFit = PipelineModel.load("pipelineFit")

# Load the saved cvModel model
cvModel = CrossValidatorModel.load("cvModel")

In [70]:
# Cassandra configuration
cassandra_contact_points = ['192.168.1.22']
cassandra_keyspace = 'stock_market' 
cassandra_table = 'coins_reddit2'

# Connect to the Cassandra cluster
cluster = Cluster(cassandra_contact_points)
session = cluster.connect()

# Switch to the keyspace
session.set_keyspace(cassandra_keyspace)

In [51]:
# Query to retrieve data from the table
query = f"SELECT id, timestamp, symbol, text FROM {cassandra_table} LIMIT 250;"

# Execute the query
result_set = session.execute(query)

In [54]:

# Create an empty list to store the rows
result_rows = []

# Define the schema for the result DataFrame
schema = StructType([
    StructField("id", StringType(), nullable=True),
    StructField("timestamp", TimestampType(), nullable=True),
    StructField("symbol", StringType(), nullable=True),
    StructField("prediction", StringType(), nullable=True),
])

# Define a mapping for the prediction values
prediction_mapping = {0.0: "Positive", 1.0: "Neutral", 2.0: "Negative"}

# Iterate over the result_set
for row in result_set:
    # Access the values of 'id', 'timestamp', and 'symbol' columns from the row
    id_value = row.id
    timestamp_value = row.timestamp
    symbol_value = row.symbol
    
    # Create a DataFrame with the new text
    new_text_df = spark.createDataFrame([Row(clean_text=row.text)])
    
    # Use the loaded pipelineFit to transform the new text data
    new_dataset = pipelineFit.transform(new_text_df)
    
    # Use the loaded cvModel to make predictions on the new text data
    new_predictions = cvModel.transform(new_dataset)
    
    # Access the 'prediction' column value from the new_predictions DataFrame
    prediction_value = new_predictions.select(col("prediction")).first()[0]
    
    # Map the prediction value to the corresponding label
    prediction_label = prediction_mapping.get(prediction_value, "Unknown")
    
    # Create a new row with 'id', 'timestamp', 'symbol', and 'prediction' values
    new_row = Row(id=id_value, timestamp=timestamp_value, symbol=symbol_value, prediction=prediction_label)
    print(new_row)
    # Add the new row to the result_rows list
    result_rows.append(new_row)

# Create a DataFrame from the result_rows list
new_dataframe = spark.createDataFrame(result_rows, schema)

# Display the new DataFrame
new_dataframe.show()

Row(id=UUID('e3701d24-c859-49e5-9922-fe49667031fb'), timestamp=datetime.datetime(2023, 12, 22, 18, 19, 16), symbol='AAPL', prediction='Neutral')
Row(id=UUID('89ec800e-8915-49a8-a4a5-4437780a07b2'), timestamp=datetime.datetime(2023, 11, 15, 13, 15, 18), symbol='AMZN', prediction='Neutral')
Row(id=UUID('fcb4db27-c0a9-4ff9-a776-7e8348044d90'), timestamp=datetime.datetime(2023, 12, 22, 18, 58, 54), symbol='AAPL', prediction='Positive')
Row(id=UUID('ab271504-ec3f-4b3d-8bfe-c745e52ce92e'), timestamp=datetime.datetime(2023, 12, 8, 1, 16, 45), symbol='GOOGL', prediction='Neutral')
Row(id=UUID('330771ed-9381-466f-ba42-0dc7607d79f7'), timestamp=datetime.datetime(2023, 12, 23, 2, 58, 7), symbol='BA', prediction='Negative')
Row(id=UUID('97f5cdb5-3ef3-40c4-9476-4e50e05b559b'), timestamp=datetime.datetime(2023, 12, 22, 14, 52, 24), symbol='AMZN', prediction='Positive')
Row(id=UUID('493211aa-45a1-45b8-9cfd-8889e12a8f59'), timestamp=datetime.datetime(2023, 12, 23, 4, 10, 25), symbol='AMZN', prediction

In [71]:
table_name_pred = "predreddit"

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name_pred} (
        timestamp TIMESTAMP PRIMARY KEY,
        symbol TEXT,
        prediction TEXT
    )
"""

session.execute(create_table_query)

for row in new_dataframe.rdd.collect():
    
    insert_query = f"""
        INSERT INTO {table_name_pred} (timestamp, symbol, prediction)
        VALUES ('{row.timestamp}', '{row.symbol}', '{row.prediction}')
    """
    session.execute(insert_query)

In [57]:
new_dataframe.select("id").first()

Row(id='{__class__=uuid.UUID, int=302316883395429105585825085843689189883}')

In [ ]:
session.shutdown()
cluster.shutdown()